#Importing Data and Libs

In [ ]:
#Pip installs
!pip install kaggle
!pip install Augmentor

#Import
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

import glob
import imageio
import numpy as np
import PIL
import time
import pathlib
from IPython import display

from zipfile import ZipFile
from google.colab import files

In [ ]:
#Kaggle Data Import
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download --force -d phylake1337/fire-dataset

file_name = "/content/fire-dataset.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Getting Data and Visualizing

In [ ]:
#figure out how to add test

import os, random
import shutil

core_dir = "content/fire_dataset/"
!mkdir -rf "content/fire_dataset/train"


fire_dir = os.path.join('/content/fire_dataset/fire_images')
length = len(os.listdir(fire))
for i in range(length, 5):
    

random.choice([x for x in os.listdir("C:\\") if os.path.isfile(os.path.join("C:\\", x))])
    shutil.move(full_path, "/content/Training/no_tumor")

!rm -rf d

In [ ]:
fire_dir = os.path.join('/content/fire_dataset/fire_images')
no_fire_dir = os.path.join('/content/fire_dataset/non_fire_images')

print('total training fire images:', len(os.listdir(fire_dir)))
print('total training no fire images:', len(os.listdir(no_fire_dir)))

fire_files = os.listdir(fire_dir)
print(fire_files[:10])

In [ ]:
from IPython.display import Image, display
display(Image('/content/fire_dataset/fire_images/fire.102.png'))

In [ ]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')

# validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/content/fire_dataset/',  
        target_size=(300, 300), 
        batch_size=128,
        class_mode='binary')

# validation_generator = validation_datagen.flow_from_directory(
#         '/content/test/',  
#         target_size=(300, 300),  
#         batch_size=128,
#         class_mode='binary')

#Actual Model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer = Adam(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

EPOCHS = 10
es = EarlyStopping(
    monitor='val_loss', 
    mode='min',
    patience=3
)

history = model.fit(
  train_generator,
  steps_per_epoch=int(999/128),  
  epochs=EPOCHS,
  # validation_data = validation_generator,
  # validation_steps=int(2000/128),
  callbacks=[es]
)

In [ ]:
model.save("wildfire-weights.h5")

#Plotting
acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.title('Training accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.title('Training loss')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
loss, acc = model.evaluate(train_generator, verbose=2)
print("Loss of model: " + str(loss)) 
print("Accuracy of model: "+ str(acc*100))